# Preparación de los datos

En este notebook se realiza las siguientes acciones de limpieza de datos sobre el conjunto de datos base:

- Cargar el conjunto de datos desde archivos .csv
- Identificar y remover duplicados.
- Identificar y remover registros con valores nulos.
- Remover variables con información sensible o fuera del alcance en futuras etapas.
- Verificaciones de calidad sobre el conjunto de datos de salida.

## 1. Importar librerías requeridas

Importar pandas y otras librerías necesarias para la manipulación y análisis de datos.

In [1]:
import pandas as pd
import numpy as np
import os
import warnings
import yaml

# Suprimir los warnings para una salida más limpia
warnings.filterwarnings('ignore')

# Opciones para una mejor visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print("¡Bibliotecas importadas con éxito!")

¡Bibliotecas importadas con éxito!


## 2. Carga del conjunto de datos

In [ ]:
# Define the path to the dataset
config = yaml.safe_load(open("../params.yaml"))["cleaning"]
dataset_path = "../" + config["input_path"]

# Check if the file exists
if os.path.exists(dataset_path):
    print(f"✓ Dataset found at: {dataset_path}")
else:
    print(f"✗ Dataset not found at: {dataset_path}")
    print("Please ensure the dataset.csv file is in the correct location.")

# Load the dataset
try:
    df = pd.read_csv(dataset_path)
    print(f"\n✓ Dataset loaded successfully!")
    print(f"Dataset shape: {df.shape}")
except Exception as e:
    print(f"✗ Error loading dataset: {e}")

✓ Dataset found at: ../data/raw/dataset.csv

✓ Dataset loaded successfully!
Dataset shape: (114000, 21)


## 3. Exploración de la estructura del conjunto de datos.

Muestra información acerca del conjunto de datos cargado como son: forma, nombres de columnas, tipos de datos y estadísticas generales.


In [3]:
# Store original dataset information for comparison
original_shape = df.shape
print(f"Original Dataset Shape: {original_shape}")
print(f"Total records: {original_shape[0]:,}")
print(f"Total features: {original_shape[1]}")
print("\n" + "="*50)

Original Dataset Shape: (114000, 21)
Total records: 114,000
Total features: 21



In [4]:
# Display first few rows
print("First 5 rows of the dataset:")
print("="*50)
display(df.head())

First 5 rows of the dataset:


,index,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [5]:
# Display dataset information
print("Dataset Information:")
print("="*50)
print(df.info())
print("\nColumn names:")
print(df.columns.tolist())

Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114000 entries, 0 to 113999
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   index             114000 non-null  int64  
 1   track_id          114000 non-null  object 
 2   artists           113999 non-null  object 
 3   album_name        113999 non-null  object 
 4   track_name        113999 non-null  object 
 5   popularity        114000 non-null  int64  
 6   duration_ms       114000 non-null  int64  
 7   explicit          114000 non-null  bool   
 8   danceability      114000 non-null  float64
 9   energy            114000 non-null  float64
 10  key               114000 non-null  int64  
 11  loudness          114000 non-null  float64
 12  mode              114000 non-null  int64  
 13  speechiness       114000 non-null  float64
 14  acousticness      114000 non-null  float64
 15  instrumentalness  114000 non-null  float64
 16 

In [6]:
# Check for missing values
print("Missing Values Analysis:")
print("="*50)
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100

missing_df = pd.DataFrame({
    'Column': missing_values.index,
    'Missing Count': missing_values.values,
    'Missing Percentage': missing_percentage.values
})

# Show only columns with missing values
missing_with_nulls = missing_df[missing_df['Missing Count'] > 0]
if len(missing_with_nulls) > 0:
    display(missing_with_nulls)
else:
    print("✓ No missing values found in the dataset!")

print(f"\nTotal missing values: {missing_values.sum():,}")

Missing Values Analysis:


,Column,Missing Count,Missing Percentage
2,artists,1,0.000877
3,album_name,1,0.000877
4,track_name,1,0.000877



Total missing values: 3


In [7]:
# Check for duplicate records
print("Duplicate Records Analysis:")
print("="*50)
duplicate_count = df.duplicated().sum()
duplicate_percentage = (duplicate_count / len(df)) * 100

print(f"Number of duplicate records: {duplicate_count:,}")
print(f"Percentage of duplicate records: {duplicate_percentage:.2f}%")

if duplicate_count > 0:
    print("\nExample of duplicate records:")
    duplicates = df[df.duplicated(keep=False)]
    display(duplicates.head(10))
else:
    print("✓ No duplicate records found!")

Duplicate Records Analysis:
Number of duplicate records: 0
Percentage of duplicate records: 0.00%
✓ No duplicate records found!


## 4. Remover registros duplicados

Identificar y remover los registros duplicados.

In [8]:
# Remove duplicate records
print("Removing Duplicate Records...")
print("="*50)

# Count duplicates before removal
duplicates_before = df.duplicated().sum()
print(f"Duplicates before removal: {duplicates_before:,}")

# Remove duplicates
df_cleaned = df.drop_duplicates()

# Count duplicates after removal
duplicates_after = df_cleaned.duplicated().sum()
records_removed = len(df) - len(df_cleaned)

print(f"Duplicates after removal: {duplicates_after:,}")
print(f"Records removed: {records_removed:,}")

if records_removed > 0:
    print(f"✓ Successfully removed {records_removed:,} duplicate records!")
else:
    print("✓ No duplicate records found to remove.")

Removing Duplicate Records...
Duplicates before removal: 0
Duplicates after removal: 0
Records removed: 0
✓ No duplicate records found to remove.
Duplicates after removal: 0
Records removed: 0
✓ No duplicate records found to remove.


## 5. Remover valores faltantes.

Identificar los registros con valores faltantes y removerlos.

In [9]:
# Remove records with missing values
print("Removing Records with Missing Values...")
print("="*50)

# Count missing values before removal
missing_before = df_cleaned.isnull().sum().sum()
records_with_nulls_before = df_cleaned.isnull().any(axis=1).sum()

print(f"Total missing values before removal: {missing_before:,}")
print(f"Records with at least one missing value: {records_with_nulls_before:,}")

# Remove records with any missing values
df_final = df_cleaned.dropna()

# Count missing values after removal
missing_after = df_final.isnull().sum().sum()
records_with_nulls_after = df_final.isnull().any(axis=1).sum()
null_records_removed = len(df_cleaned) - len(df_final)

print(f"\nTotal missing values after removal: {missing_after:,}")
print(f"Records with missing values after removal: {records_with_nulls_after:,}")
print(f"Records removed due to missing values: {null_records_removed:,}")

if null_records_removed > 0:
    print(f"✓ Successfully removed {null_records_removed:,} records with missing values!")
else:
    print("✓ No records with missing values found to remove.")

Removing Records with Missing Values...


Total missing values before removal: 3
Records with at least one missing value: 1

Total missing values after removal: 0
Records with missing values after removal: 0
Records removed due to missing values: 1
✓ Successfully removed 1 records with missing values!

Total missing values after removal: 0
Records with missing values after removal: 0
Records removed due to missing values: 1
✓ Successfully removed 1 records with missing values!


# 6. Remover varaibles que no serán usadas.

In [10]:
# Remove specified columns before saving
columns_to_remove = ['index', 'track_id', 'artists', 'album_name', 'track_name']
print(f"Removing columns: {columns_to_remove}")

# Check which columns exist in the dataset
existing_columns_to_remove = [col for col in columns_to_remove if col in df_final.columns]
missing_columns = [col for col in columns_to_remove if col not in df_final.columns]

if existing_columns_to_remove:
    print(f"Columns found and will be removed: {existing_columns_to_remove}")
    df_final_for_saving = df_final.drop(columns=existing_columns_to_remove)
    print(f"✓ Successfully removed {len(existing_columns_to_remove)} columns")
else:
    df_final_for_saving = df_final.copy()
    print("✓ No specified columns found to remove")

if missing_columns:
    print(f"Columns not found in dataset: {missing_columns}")

print(f"Shape after column removal: {df_final_for_saving.shape}")
print(f"Remaining columns: {list(df_final_for_saving.columns)}")

Removing columns: ['index', 'track_id', 'artists', 'album_name', 'track_name']
Columns found and will be removed: ['index', 'track_id', 'artists', 'album_name', 'track_name']
✓ Successfully removed 5 columns
Shape after column removal: (113999, 16)
Remaining columns: ['popularity', 'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'track_genre']


## 7. Revisar los resultados de la limpieza.

Muestra el resultados final de la limpieza y compara el antes y después para verificar que haya sido existosa.

In [11]:
# Create comprehensive cleaning summary
print("DATA CLEANING SUMMARY")
print("="*60)

# Calculate total records removed
total_records_removed = len(df) - len(df_final)
percentage_removed = (total_records_removed / len(df)) * 100
percentage_retained = ((len(df_final)) / len(df)) * 100

print(f"Original dataset shape: {df.shape}")
print(f"Final dataset shape: {df_final.shape}")
print(f"\nRecords removed: {total_records_removed:,} ({percentage_removed:.2f}%)")
print(f"Records retained: {len(df_final):,} ({percentage_retained:.2f}%)")

print(f"\nBreakdown of records removed:")
print(f"  - Duplicate records: {records_removed:,}")
print(f"  - Records with missing values: {null_records_removed:,}")
print(f"  - Total removed: {total_records_removed:,}")

DATA CLEANING SUMMARY
Original dataset shape: (114000, 21)
Final dataset shape: (113999, 21)

Records removed: 1 (0.00%)
Records retained: 113,999 (100.00%)

Breakdown of records removed:
  - Duplicate records: 0
  - Records with missing values: 1
  - Total removed: 1


In [12]:
# Verify final dataset quality
print("FINAL DATASET QUALITY CHECK")
print("="*60)

# Check for duplicates in final dataset
final_duplicates = df_final.duplicated().sum()
print(f"Duplicate records in final dataset: {final_duplicates:,}")

# Check for missing values in final dataset
final_missing = df_final.isnull().sum().sum()
print(f"Missing values in final dataset: {final_missing:,}")

# Data quality status
if final_duplicates == 0 and final_missing == 0:
    print("\n✅ DATA CLEANING SUCCESSFUL!")
    print("✓ No duplicate records")
    print("✓ No missing values")
    print("✓ Dataset is ready for analysis and modeling")
else:
    print("\n⚠️ WARNING: Data cleaning may not be complete")
    if final_duplicates > 0:
        print(f"⚠️ Still {final_duplicates:,} duplicate records remain")
    if final_missing > 0:
        print(f"⚠️ Still {final_missing:,} missing values remain")

FINAL DATASET QUALITY CHECK
Duplicate records in final dataset: 0
Missing values in final dataset: 0

✅ DATA CLEANING SUCCESSFUL!
✓ No duplicate records
✓ No missing values
✓ Dataset is ready for analysis and modeling


## 8. Guardar el conjunto de datos limpio.

In [ ]:
# Optional: Save the cleaned dataset
print("SAVE CLEANED DATASET")
print("="*60)

# Create output directory if it doesn't exist
output_dir = "../" + config["output_path"]
os.makedirs(output_dir, exist_ok=True)

# Save the cleaned dataset
output_path = os.path.join(output_dir, 'dataset_cleaned.csv')
df_final_for_saving.to_csv(output_path, index=False)

print(f"\n✓ Cleaned dataset saved to: {output_path}")
print(f"✓ File size: {os.path.getsize(output_path):,} bytes")
print(f"✓ Records saved: {len(df_final_for_saving):,}")
print(f"✓ Features saved: {len(df_final_for_saving.columns)}")

print("\n🎉 Data preprocessing completed successfully!")

SAVE CLEANED DATASET

✓ Cleaned dataset saved to: ../data/prep/dataset_cleaned.csv
✓ File size: 10,058,238 bytes
✓ Records saved: 113,999
✓ Features saved: 16

🎉 Data preprocessing completed successfully!

✓ Cleaned dataset saved to: ../data/prep/dataset_cleaned.csv
✓ File size: 10,058,238 bytes
✓ Records saved: 113,999
✓ Features saved: 16

🎉 Data preprocessing completed successfully!
